In [1]:
bind "set show-mode-in-prompt off"  # Turn off showing the vi mode in prompt, which clutters up the output here

In [65]:
# Function for linking to file in companion repo
repo-link() {
echo "\`\`\`\`{margin}
\`\`\`{admonition} Repository link
:class: seealso
See \`$(basename $1)\` in the companion repo [here](https://github.com/edsq/eeskew-pwg-test-000/tree/$(git rev-parse HEAD)/$1).
\`\`\`
\`\`\`\`" | displayMD
}

# Alias for colored ls
alias ls="gls --color=always"

# Project Management with PDM

Part 1 of notes from talk given to the WSU Python Working Group on February 8, 2023.

Here I show the basics of project management with [PDM](https://pdm.fming.dev/latest/): how to create a project,
add dependencies, and add development dependencies.

This tutorial is primarily aimed at macOS and Linux users, although the commands for Windows should mostly translate.

:::{note}
A companion repository with the example project created in these notes is available [here](https://github.com/edsq/eeskew-pwg-test-000).
:::

Requirements:

- The ability to get python executables of different versions, such as with [pyenv](https://github.com/pyenv/pyenv) or [conda](https://docs.conda.io/en/latest/miniconda.html)
- [PDM](https://pdm.fming.dev/latest/) available globally
- The [pdm-bump](https://github.com/carstencodes/pdm-bump) plugin installed

First, create the project directory and `cd` into it:

```bash
mkdir eeskew-pwg-test-000
cd eeskew-pwg-test-000
```

:::{important}
Because this is a throwaway test project, it is important that you give your project a name that won't conflict with any other package on PyPI or TestPyPI. Adding your name and some numbers is a good way to ensure this.
:::

## Set the python version and initialize the project

Here, we'll use python version 3.11, but you may change this to be whatever you like.  I'll cover two methods of setting the python version: using [`pyenv`](https://github.com/pyenv/pyenv), and using [`conda`](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

### Using `pyenv`

Install python 3.11 if it is not already (see installed versions with `pyenv versions`):

```bash
pyenv install 3.11
```

Set the local python version for this project and initialize using that version:

```bash
pyenv local 3.11
pdm init --python python
```

`pyenv local` creates a file `.python-version`, which `pyenv` reads and redirects the command `python` to the installed `python3.11`.  Thus, we only need to tell pdm to use the usual `python` executable.

### Using `conda`

PDM can also use `conda` to create your virtual environment.  To do this simply, we create the virtual environment before initializing the project, so that we can pass the right python executable to `pdm init`.

```bash
pdm venv create -w conda 3.11
pdm init --python .venv/bin/python
```

::::{tab-set}
:::{tab-item} Using `pyenv`

Install python 3.11 if it is not already (see installed versions with `pyenv versions`):

```bash
pyenv install 3.11
```

Set the local python version for this project and initialize using that version:

```bash
pyenv local 3.11
pdm init --python python
```

`pyenv local` creates a file `.python-version`, which `pyenv` reads and redirects the command `python` to the installed `python3.11`.  Thus, we only need to tell pdm to use the usual `python` executable.
:::

:::{tab-item} Using `conda`

PDM can use `conda` to create your virtual environment.  To do this simply, we create the virtual environment before initializing the project, so that we can pass the right python executable to `pdm init`.

```bash
pdm venv create -w conda 3.11
pdm init --python .venv/bin/python
```
:::
::::

### `pdm init` options

For

```
Is the project a library that is installable?
A few more questions will be asked to include a project name and build backend [y/n] (n):
```

select `y`.  Otherwise, all the default options should be good.

In [3]:
# Make sure we're in the repo directory
# Note: if cwd is already the companion repo, this will fail.
# Either restart the kernel first or don't run this cell, in that case.
project_dir=$(pdm info --where)
cd $project_dir/repos/eeskew-pwg-test-000

In [4]:
# Start companion repo fresh from the beginning
git clean -dfx  # remove all untracked files (src, build, dist, .venv)
git checkout $(git rev-list --topo-order main | tail -1)  # check out first commit
pdm venv create --force python

Removing .venv/
HEAD is now at 1efdf5a pdm init skeleton
⠴ Creating virtualenv using virtualenv.....
Virtualenv /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/.venv is created
successfully


In [5]:
# This cell hidden in presentation and docs
# Check that the environment and project are correct
pdm info

PDM version:
  2.4.8
Python Interpreter:
  /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/.venv/bin/python (3.11)
Project Root:
  /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000
Project Packages:
  None


## The PDM project

Let's take a look at what we've created:

In [6]:
ls -a

.   .git        .pdm.toml        .venv      pyproject.toml
..  .gitignore  .python-version  README.md


The relevant files created are the `README.md`; `.pdm.toml`, which holds local configuration for this PDM project; and `pyproject.toml`, which holds project tool configuration and package metadata.

:::{seealso}
PDM has also created a virtual environment for us in the `.venv` directory.  This is where our package and its dependencies will be installed.  If you are unfamiliar with virtual environments, I recommend [this article](https://realpython.com/python-virtual-environments-a-primer/).
:::

:::{note}
PDM supports the draft Python Enhancement Proposal (PEP) [582](https://peps.python.org/pep-0582/).  An alternative to virtual environments, PEP 582 would automatically get dependencies from a `__pypackages__` directory in the project root, without having to activate a virtualenv.  See the PDM docs on PEP 582 [here](https://pdm.fming.dev/latest/usage/pep582/).  We will not be using PEP 582 mode in this tutorial.
:::

### The `pyproject.toml` file

In [8]:
FILE="pyproject.toml"
echo "\`\`\`toml
# $FILE
$(cat $FILE)
\`\`\`" | displayMD

```toml
# pyproject.toml
[tool.pdm]

[project]
name = "eeskew-pwg-test-000"
version = "0.1.0"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = []
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
```


This file is written in `.toml` format, which stands for [Tom's Obvious Minimal Language](https://toml.io/en/).

The `tool.pdm` table is empty, although we'll add things here later on.

The `project` table contains the metadata needed to install our project.  Its values thus far were set by the options we chose while running `pdm init`.

The `build-system` section tells the build frontend (e.g. `pip`) what build backend to use - the build backend is what will actually create the distribution artifacts (wheels and sdists), which we'll see later.  See [PEP 517](https://peps.python.org/pep-0517/) for more information.  Here, we're just using the default PDM backend.

:::{seealso}
See the [PDM docs on writing `pyproject.toml`](https://pdm.fming.dev/latest/pyproject/pep621/) for more on what can be specified in this file.
:::

## Adding code

First we create our package directory in [src layout](https://hynek.me/articles/testing-packaging/), with an empty (for now) `__init__.py` file to indicate that it is a python package:

In [9]:
mkdir src
mkdir src/eeskew_pwg_test_000
touch src/eeskew_pwg_test_000/__init__.py

:::{note}
By convention, we use [snake_case](https://en.wikipedia.org/wiki/Snake_case) for the package name, while we used [kebab-case](https://en.wiktionary.org/wiki/kebab_case) for the repository name.
:::

In [10]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 1efdf5a pdm init skeleton
HEAD is now at a10844f Add __init__.py


### Add a module

Let's add some code in `src/eeskew_pwg_test_000/utils.py`:

In [11]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out
EOF

In [12]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was a10844f Add __init__.py
HEAD is now at 229305b Add utils.py


In [55]:
repo-link src/eeskew_pwg_test_000/utils.py

````{margin}
```{seealso}
See this file in the companion repo [here](https://github.com/edsq/eeskew-pwg-test-000/tree/5688f464ab30b7c01070d0956394abb8c7fbcf70/src/eeskew_pwg_test_000/utils.py).
```
````


In [14]:
FILE="src/eeskew_pwg_test_000/utils.py"
echo "\`\`\`python
# $FILE
$(cat $FILE)
\`\`\`" | displayMD

```python
# src/eeskew_pwg_test_000/utils.py
def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out
```


The actual content of this code is not too important for the purposes of these notes, but for completeness, all it does is capitalize and lowercase alternating letters in a string.

## Install the project

To make our code available in the virtual environment, we have to install it:

In [15]:
pdm install

Lock file does not exist
Updating the lock file...
⠋ Fetching hashes for resolved packages...
🔒 Lock successful
Changes are written to pdm.lock.
⠋ Fetching hashes for resolved packages...
All packages are synced to date, nothing to do.
Installing the project as an editable package...
  ✔ Install eeskew-pwg-test-000 0.1.0 successful
36m⠼ Installing eeskew-pwg-test-000 0.1.0...
🎉 All complete!
36m⠼ Installing eeskew-pwg-test-000 0.1.0...



In [16]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 229305b Add utils.py
HEAD is now at e12db04 Add pdm.lock after first pdm install


Now we can import our package:

In [17]:
pdm run python -c 'from eeskew_pwg_test_000.utils import sarcasm; print(sarcasm("Hello world!"))'

hElLo wOrLd!


Note we have to type `pdm run` before our command for it to be run within our project environment.

:::{tip}
If you don't want to type `pdm run` every time before a command to be run in the project virtual environment, you can *activate* the environment, which will modify your `sys.prefix` to point to the `.venv` directory.  See the python `venv` docs [here](https://docs.python.org/3/library/venv.html#how-venvs-work) for more on how virtual environments work.

`pdm` will print the command to activate the project virtual environment with command `pdm venv activate`.  You can copy and paste that output, or, if you want to activate the environment in one line, use:

```bash
eval $(pdm venv activate)
```

To simplify things further, add this as an alias to your `~/.bashrc` or `~/.bash_profile` (and don't forget to restart your shell or `source ~/.bashrc` after):

```bash
# ~/.bashrc
alias pdm-activate='eval $(pdm venv activate)'
```

This will let you activate the environment with the command `pdm-activate`.

You can deactivate an active virtual environment with the command `deactivate`.  See also the [PDM docs on virtualenv activation](https://pdm.fming.dev/latest/usage/venv/#activate-a-virtualenv).
:::

### The `pdm.lock` file

Running `pdm install` also created a new file, `pdm.lock`:

In [18]:
ls

README.md  pdm.lock  pyproject.toml  src


In [19]:
FILE="pdm.lock"
echo "\`\`\`toml
# $FILE
$(cat pdm.lock)
\`\`\`" | displayMD

```toml
# pdm.lock
# This file is @generated by PDM.
# It is not intended for manual editing.

[metadata]
lock_version = "4.1"
content_hash = "sha256:86165c41f17b4b263a688544a3ebc55eccc1713dd177c40649b2e936dab66751"

[metadata.files]
```


This is a lockfile, which will contain the exact versions of each project dependency we install.  It is useful for creating a perfect reproduction of the project virtual environment, which keeps our development reproducible over time and across different machines.

Right now, we have not installed anything other than the project itself, so it is essentially empty.

## Add a dependency

Let's add a dependency to our project:

In [20]:
pdm add cowsay

Adding packages to default dependencies: cowsay
⠙ Fetching hashes for resolved packages...3m5.0
🔒 Lock successful
Changes are written to pdm.lock.
Changes are written to pyproject.toml.
⠋ Fetching hashes for resolved packages...
Synchronizing working set with lock file: 1 to add, 0 to update, 0 to remove

  ✔ Install cowsay 5.0 successful
Installing the project as an editable package...0m...
  ✔ Update eeskew-pwg-test-000 0.1.0 -> 0.1.0 successful
36m⠙ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...
🎉 All complete!
36m⠙ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...



In [21]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was e12db04 Add pdm.lock after first pdm install
HEAD is now at e5e9b85 Add cowsay to dependencies


### What did `pdm add` do?

`cowsay` now appears as a dependency in `pyproject.toml`:

In [22]:
echo "\`\`\`toml
$(simple-diff HEAD~ pyproject.toml)
\`\`\`" | displayMD

```toml
# pyproject.toml
...

authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}

...
```


Click below to show diff:

In [23]:
git diff --color HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:10 @ description = "A test project for presentation to the WSU Python Working Group."
 authors = [
     {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
 ]
-dependencies = []
+dependencies = [
+    "cowsay>=5.0",
+]
 requires-python = ">=3.11"
 readme = "README.md"
 license = {text = "MIT"}


We've also updated `pdm.lock` to include cowsay:

In [24]:
echo "\`\`\`toml
$(simple-diff HEAD~ pdm.lock)
\`\`\`" | displayMD

```toml
# pdm.lock
# This file is @generated by PDM.
# It is not intended for manual editing.

[[package]]
name = "cowsay"
version = "5.0"
summary = "The famous cowsay for GNU/Linux is now available for python"

[metadata]
lock_version = "4.1"
content_hash = "sha256:a04c8eb9409090bc9acb94e5cec5ef19afdecacb2c169628142af5ca472135f0"

[metadata.files]
"cowsay 5.0" = [
    {url = "https://files.pythonhosted.org/packages/6b/b8/9f497fd045d74fe21d91cbe8debae0b451229989e35b539d218547d79fc6/cowsay-5.0.tar.gz", hash = "sha256:c00e02444f5bc7332826686bd44d963caabbaba9a804a63153822edce62bbbf3"},
]
```


We can now import `cowsay`:

In [25]:
pdm run python -c 'import cowsay; cowsay.cow("moo!")'

  ____
| moo! |
  ====
    \
     \
       ^__^
       (oo)\_______
       (__)\       )\/\
           ||----w |
           ||     ||


:::{seealso}
See the [PDM docs on managing dependencies](https://pdm.fming.dev/latest/usage/dependency/) for more information.
:::

### Adding more code

Let's add a new function to `utils.py`:

In [26]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
import cowsay

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out

def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)
EOF

In [27]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was e5e9b85 Add cowsay to dependencies
HEAD is now at fe97875 Add sarcastic_cowsay to utils


In [28]:
repo-link src/eeskew_pwg_test_000/utils.py

````{margin}
    ```{seealso}
    See this file in the companion repo [here](https://github.com/edsq/eeskew-pwg-test-000/tree/fe978753a6b60bbb5eaba1333b21be265e6a3e3b/src/eeskew_pwg_test_000/utils.py).
    ```
    ````


In [29]:
echo "\`\`\`python
$(simple-diff --context 0 HEAD~ src/eeskew_pwg_test_000/utils.py)
\`\`\`" | displayMD

```python
# src/eeskew_pwg_test_000/utils.py
import cowsay


...


def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)
```


Click below to show diff:

In [30]:
git diff --color HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:2 @
+import cowsay
+
 def sarcasm(s):
     """Convert string `s` to sArCaSm TeXt."""
     out = ""
@ src/eeskew_pwg_test_000/utils.py:14 @ def sarcasm(s):
             out += c.upper()
 
     return out
+
+def sarcastic_cowsay(s):
+    """Cowsay `s`, sArCaStIcAlLy."""
+    sarcastic_s = sarcasm(s)
+    cowsay.cow(sarcastic_s)


We can now run this new function:

In [31]:
pdm run python -c 'from eeskew_pwg_test_000.utils import sarcastic_cowsay; sarcastic_cowsay("mooo!")'

  _____
| mOoO! |
  =====
     \
      \
        ^__^
        (oo)\_______
        (__)\       )\/\
            ||----w |
            ||     ||


:::{note}
We didn't have to re-run `pdm install` to use our new function - this is because PDM installs our `eeskew_pwg_test_000` package in "editable mode", which acts sort of like a symlink between the source code and the installed files in the `.venv` directory.
:::

## Add a development dependency

The dependencies listed in the `project.dependencies` section of `pyproject.toml` will all be installed when someone runs `pip install eeskew_pwg_test_project`.  What if we have dependencies we only want in our development environment?

Let's add `black`, a tool to automatically format our code:

In [32]:
pdm add -d black

Adding packages to dev dev-dependencies: black
⠋ Fetching hashes for resolved packages...m23.1.0
🔒 Lock successful
Changes are written to pdm.lock.
Changes are written to pyproject.toml.
⠋ Fetching hashes for resolved packages...
Synchronizing working set with lock file: 6 to add, 0 to update, 0 to remove

  ⠋ Installing black 23.1.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing packaging 23.0...       
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing packaging 23.0...       
  ⠋ Installing pathspec 0.11.0...      
  ⠙ Installing black 23.1.0...         
  ⠙ Installing click 8.1.3...          
  ⠙ Installing mypy-e

In [33]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was fe97875 Add sarcastic_cowsay to utils
HEAD is now at 0424f4c Add black to dev dependencies


### What happened?

We've added a new `[dev-dependencies]` sub-table to the `[tool.pdm]` table, which contains `black`:

In [34]:
echo "\`\`\`toml
$(simple-diff HEAD~ pyproject.toml)
\`\`\`" | displayMD

```toml
# pyproject.toml
[tool.pdm]
[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"

...
```


Click below to show diff:

In [35]:
git diff --color HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:4 @
 [tool.pdm]
+[tool.pdm.dev-dependencies]
+dev = [
+    "black>=23.1.0",
+]
 
 [project]
 name = "eeskew-pwg-test-000"


When we run `pdm install`, by default, PDM will install dependencies from here in addition to the dependencies listed in `[project.dependencies]`.  A different tool like `pip`, however, will not.

:::{seealso}
See the [PDM docs on adding development dependencies](https://pdm.fming.dev/latest/usage/dependency/#add-development-only-dependencies) or the [PDM docs on development dependencies in `pyproject.toml`](https://pdm.fming.dev/latest/pyproject/tool-pdm/#development-dependencies) for more information on development dependencies.
:::

The lockfile has also been updated:

In [36]:
cat pdm.lock

# This file is @generated by PDM.
# It is not intended for manual editing.

[[package]]
name = "black"
version = "23.1.0"
requires_python = ">=3.7"
summary = "The uncompromising code formatter."
dependencies = [
    "click>=8.0.0",
    "mypy-extensions>=0.4.3",
    "packaging>=22.0",
    "pathspec>=0.9.0",
    "platformdirs>=2",
]

[[package]]
name = "click"
version = "8.1.3"
requires_python = ">=3.7"
summary = "Composable command line interface toolkit"
dependencies = [
    "colorama; platform_system == \"Windows\"",
]

[[package]]
name = "colorama"
version = "0.4.6"
requires_python = "!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*,!=3.5.*,!=3.6.*,>=2.7"
summary = "Cross-platform colored terminal text."

[[package]]
name = "cowsay"
version = "5.0"
summary = "The famous cowsay for GNU/Linux is now available for python"

[[package]]
name = "mypy-extensions"
version = "1.0.0"
requires_python = ">=3.5"
summary = "Type system extensions for programs checked with the mypy type checker."

[[package

    {url = "https://files.pythonhosted.org/packages/c2/f1/df59e28c642d583f7dacffb1e0965d0e00b218e0186d7858ac5233dce840/click-8.1.3-py3-none-any.whl", hash = "sha256:bb4d8133cb15a609f44e8213d9b391b0809795062913b383c62be0ee95b1db48"},
]
"colorama 0.4.6" = [
    {url = "https://files.pythonhosted.org/packages/d1/d6/3965ed04c63042e047cb6a3e6ed1a63a35087b6a609aa3a15ed8ac56c221/colorama-0.4.6-py2.py3-none-any.whl", hash = "sha256:4f1d9991f5acc0ca119f9d443620b77f9d6b33703e51011c16baf57afb285fc6"},
    {url = "https://files.pythonhosted.org/packages/d8/53/6f443c9a4a8358a93a6792e2acffb9d9d5cb0a5cfd8802644b7b1c9a02e4/colorama-0.4.6.tar.gz", hash = "sha256:08695f5cb7ed6e0531a20572697297273c47b8cae5a63ffc6d6ed5c201be6e44"},
]
"cowsay 5.0" = [
    {url = "https://files.pythonhosted.org/packages/6b/b8/9f497fd045d74fe21d91cbe8debae0b451229989e35b539d218547d79fc6/cowsay-5.0.tar.gz", hash = "sha256:c00e02444f5bc7332826686bd44d963caabbaba9a804a63153822edce62bbbf3"},
]
"mypy-extensions 1.0.0" = [
    {ur

Many new packages now exist in the lockfile, not just `cowsay` and `black`.  This is because (unlike `cowsay`), `black` itself has dependencies that we needed to install to get it to work.  This lock file records exactly the versions of those sub-dependencies that we've now installed into our project virtual environment.

When we run `pdm install`, if the lock file exists (and `pyproject.toml` hasn't been changed since the lockfile was last updated), PDM will install precisely the packages listed in the lockfile, so we'll always be working in the same virtual environment.  This is useful for developing and testing the code, so you should always include the lockfile in your project version control.

However, we don't want to impose these restrictions on users of our library, or our project would rapidly become impossible to install due to other packages requiring different versions of the packages in the lockfile.  The only thing that we care about is that users have the right versions of the dependencies we directly use, which are listed in the `project.dependencies` array in `pyproject.toml`.  This is why `pip install` does not care about the existence of the lockfile.

:::{seealso}
See the [PDM docs on version control](https://pdm.fming.dev/latest/usage/project/#working-with-version-control) for more on best practices for version-controlling a PDM project.
:::

### Using black

We can now run `black` within our environment.  Let's re-write our code with poor formatting (note the spacing around the `==`, `%`, and `+=` operators), and then run `black` on it:

In [37]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
import cowsay

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ''
    for i,c in enumerate(s):
        if i% 2 ==0:
            out +=c.lower()

        else:
            out+= c.upper()

    return out

def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)
EOF

In [38]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 0424f4c Add black to dev dependencies
HEAD is now at 66ceb46 Use bad formatting in utils for black to fix


In [39]:
repo-link src/eeskew_pwg_test_000/utils.py

````{margin}
    ```{seealso}
    See this file in the companion repo [here](https://github.com/edsq/eeskew-pwg-test-000/tree/66ceb46848f4bdc087b21f251911bdd41b42a851/src/eeskew_pwg_test_000/utils.py).
    ```
    ````


In [40]:
echo "\`\`\`python
$(simple-diff --context 2 HEAD~ src/eeskew_pwg_test_000/utils.py)
\`\`\`" | displayMD

```python
# src/eeskew_pwg_test_000/utils.py
...

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ''
    for i,c in enumerate(s):
        if i% 2 ==0:
            out +=c.lower()

        else:
            out+= c.upper()

    return out

...
```


Click below to show diff:

In [41]:
git diff --color HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:5 @ import cowsay
 
 def sarcasm(s):
     """Convert string `s` to sArCaSm TeXt."""
-    out = ""
-    for i, c in enumerate(s):
-        if i % 2 == 0:
-            out += c.lower()
+    out = ''
+    for i,c in enumerate(s):
+        if i% 2 ==0:
+            out +=c.lower()
 
         else:
-            out += c.upper()
+            out+= c.upper()
 
     return out
 


This is bad formatting!  Rather than fix it manually, we can run `black` on our code, which will auto-impose a reasonable style.

In [42]:
pdm run black src/

reformatted /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/src/eeskew_pwg_test_000/utils.py

All done! ✨ 🍰 ✨
1 file reformatted, 1 file left unchanged.


In [43]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 66ceb46 Use bad formatting in utils for black to fix
HEAD is now at 5688f46 Fix formatting with black


### What happened?

In [39]:
repo-link src/eeskew_pwg_test_000/utils.py

````{margin}
    ```{seealso}
    See this file in the companion repo [here](https://github.com/edsq/eeskew-pwg-test-000/tree/66ceb46848f4bdc087b21f251911bdd41b42a851/src/eeskew_pwg_test_000/utils.py).
    ```
    ````


In [44]:
git diff --color HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:4 @
 import cowsay
 
+
 def sarcasm(s):
     """Convert string `s` to sArCaSm TeXt."""
-    out = ''
-    for i,c in enumerate(s):
-        if i% 2 ==0:
-            out +=c.lower()
+    out = ""
+    for i, c in enumerate(s):
+        if i % 2 == 0:
+            out += c.lower()
 
         else:
-            out+= c.upper()
+            out += c.upper()
 
     return out
 
+
 def sarcastic_cowsay(s):
     """Cowsay `s`, sArCaStIcAlLy."""
     sarcastic_s = sarcasm(s)


`black` has automatically re-formatted our code, fixing the poor formatting we introduced earlier.

:::{seealso}
See the [black documentation](https://black.readthedocs.io/en/stable/) for more information.
:::

## Conclusion

That's all for Part 1!  The examples here should be enough to get you started using PDM to manage your own python projects.  For more advanced usage, check out the PDM documentation on:

- [General project management](https://pdm.fming.dev/latest/usage/project/)
- [Working with virtualenv](https://pdm.fming.dev/latest/usage/venv/)
- [Managing dependencies](https://pdm.fming.dev/latest/usage/dependency/)

In Part 2, we'll cover how to publish a PDM project on PyPI, so that it can be installed with a simple `pip install`.